In [289]:
import simpy 
import random
import math
import numpy as np

In [290]:
#Numero de Rampas y Fitters. Ademas se define a joe como recurso (igual a 1)
ramps = 3
fitters = 2
joes = 1

In [291]:
# Tiempo de simulacion y demas constantes/variables (en minutos)
#Maximo 4 carros en fila o se van los que llegan
simulationTime = 600
maxQueueInspection = 4
queueInspection = 0
timeQueueInspection = 0
averagetimeperclient = 0
tottim = 0
totalTime = 0
clientesAfterInspection = 0
clientesAfterService = 0
totalclientes = 0

In [292]:
#Tiempo de Inspeccion: Distribucion normal con media 5 y desviacion estandar de 1.5 (minutos)
def inspection(env, cliente):
    time = random.normalvariate(5,1.5)
    yield env.timeout(time)
    print(cliente + ' car inspection: %3.f minutes.' %(time))

#Tiempo de fitting: Distribucion lognormal con media 40 y desviacion estandar 10 (minutos)
def fitting(env, cliente):
    time = random.normalvariate(40,10)
    yield env.timeout(time)
    print(cliente + ' fitting: %3.f minutes.' %(time))

#Segunda inspeccion (Por parte de Joe)
def secondInspection(env, cliente):
    time = random.uniform(1,3)
    yield env.timeout(time)
    print(cliente + ' second inspection: %3.f minutes.' %(time))

#Tiempo de pago: Distribucion exponencial media de 3 minutos
def payment(env, cliente):
    time = random.expovariate(1/3)
    yield env.timeout(time)
    print(cliente + ' payment in %3.f minutes.' %(time))

In [293]:
#Generacion de la llegada de los clientes al parlour
#Distribucion exponencial con media de 15 minutos
def llegadasClientes (env, ramp, fitter, joe):
  cont = 0
  while True:
    arrivalTime = random.expovariate(1/15)
    yield env.timeout(arrivalTime)
    cont+=1
    client= 'Cliente ' + str(cont)
    env.process(customers(env, client, ramp, fitter, joe))

In [294]:
# Metodo global, toma el environment y los recursos del problema
def customers(env, cliente, ramp, fitter, joe):
  global simulationTime,tottim, totalTime, clientesAfterInspection, clientesAfterService, queueInspection, maxQueueInspection, totalclientes

#Tiempo de llegada del clientes
  arrivalTime=env.now
  totalclientes += 1
  print(cliente +' arrived: minute %.2f .' %(arrivalTime)) 
  
#Solo entra a la fila si hay menos de 4 carros, si no simplemente de va del parlour
  if queueInspection >= maxQueueInspection:
    print(cliente + ' left. Line was full:  %.2f minutes' %(env.now))
  else:
# Entra al parlour en donde utiliza una rampa y a Joe
# En caso de haber fila, hace fila y luego pasa a la inspeccion
    queueInspection += 1
    with ramp.request() as useRamp:
      yield useRamp
      with joe.request() as useJoe:
        yield useJoe
        queueInspection = queueInspection - 1
        waitForInspectionTime = env.now - arrivalTime
        if waitForInspectionTime == 0:
          print(cliente + ' inspection: NO LINE')
        else:
          print(cliente + ' inspection: %.2f minutes in line. <-----------------' %(waitForInspectionTime))
        yield env.process(inspection(env, cliente))
        joe.release(useJoe)
      ramp.release(useRamp)
    #Tiempo final de la primera inspeccion
    endOfInspectionTime = env.now

#Luego de dar el consejo inicial por parte de Joe, el 30% de los clientes se van y el 70% se quedan
#Si hace parte del 70% para al fitting donde utiliza una rampa y un fitter (Joe ya esta libre aca)
    abandon = random.uniform(0,1)
    if abandon < 0.7:
      while True:
        with ramp.request() as useRamp:
          yield useRamp
          with fitter.request() as useFitter:
            yield useFitter
            waitInQueueTimeForFitting = env.now - endOfInspectionTime
            if waitInQueueTimeForFitting == 0:
              print( cliente +' fitting: NO LINE')
            else:
              print(cliente + ' waited %.2f minutes in queue for fitting.' %(waitInQueueTimeForFitting))
            yield env.process(fitting(env, cliente))
            fitter.release(useFitter)
          endOfFittingTime = env.now
#Tiempo final de fitting
#Se utiliza a joe para ls "Segunda" Inspeccion
          with joe.request() as useJoe:
            yield useJoe
            waitInQueueTimeForInspection = env.now - endOfFittingTime
            if waitInQueueTimeForInspection == 0:
              print( cliente +' inspection: NO LINE')
            else:
              print(cliente + ' waited %.2f minutes in queue for second inspection.' %(waitInQueueTimeForInspection))
            yield env.process(secondInspection(env, cliente))
            joe.release(useJoe)
          ramp.release(useFitter)
#10% de las veces, el trabajo no es satisfactorio y debe de hacerse un refitting. 
#Si esta en el 90% de los clientees simplemente pasa al payment
          refitting = random.uniform(0,1)
          if refitting <0.9:
            break
          else:
            print(cliente+' has to go through refitting.')
    
      endTime = env.now
#Se utiliza a Joe para el pago luego de pasar por las inspecciones y el fitting
      with joe.request() as useJoe:
        yield useJoe
        waitInQueueTimeForPayment = env.now - endTime
        if waitInQueueTimeForPayment == 0:
          print( cliente +' did not have to queue for payment.')
        else:
          print(cliente + ' waited %.2f minutes in queue for payment.' %(waitInQueueTimeForPayment))
        yield env.process(payment(env, cliente))
        joe.release(useJoe)  

#El clientee termina el ciclo y sale del parlour
      exitTime = env.now
      totalTime = exitTime - arrivalTime
      print(cliente +' leaves parlour: minute %.2f' %(env.now))
      print('----------------------------------> Total time %.2f' %(totalTime))
      clientesAfterService += 1
      tottim += totalTime

    else: 
    #Aca es si abandono despues de la primera inspeccion (30%)
      print(cliente + ' abandoned parlour: minute %.2f' %(env.now))
      clientesAfterInspection += 1




In [295]:
#De aca corremos el proceso de llega de clientes y todo el enviroment con el tiempo de simulacion y los recursos definidos
def main():
  print('Start of simulation')
  env = simpy.Environment()
  a = 0.2 #choose a good random seed
  random.seed(a)
#Definicion de los recursos
  fitter = simpy.Resource(env, fitters)
  ramp = simpy.Resource(env, ramps)
  joe = simpy.Resource(env, joes)
  env.process(llegadasClientes(env, ramp, fitter, joe))
  env.run(until = simulationTime)


In [296]:
main()

Start of simulation
Cliente 1 arrived: minute 3.13 .
Cliente 1 inspection: NO LINE
Cliente 2 arrived: minute 3.27 .
Cliente 3 arrived: minute 7.94 .
Cliente 1 car inspection:   5 minutes.
Cliente 1 abandoned parlour: minute 8.14
Cliente 2 inspection: 4.88 minutes in line. <-----------------
Cliente 2 car inspection:   5 minutes.
Cliente 2 abandoned parlour: minute 13.05
Cliente 3 inspection: 5.11 minutes in line. <-----------------
Cliente 3 car inspection:   5 minutes.
Cliente 3 fitting: NO LINE
Cliente 4 arrived: minute 22.68 .
Cliente 4 inspection: NO LINE
Cliente 5 arrived: minute 23.77 .
Cliente 4 car inspection:   5 minutes.
Cliente 5 inspection: 3.55 minutes in line. <-----------------
Cliente 4 fitting: NO LINE
Cliente 5 car inspection:   4 minutes.
Cliente 6 arrived: minute 33.70 .
Cliente 3 fitting:  23 minutes.
Cliente 3 inspection: NO LINE
Cliente 5 waited 10.61 minutes in queue for fitting.
Cliente 7 arrived: minute 41.78 .
Cliente 3 second inspection:   3 minutes.
Cliente

In [297]:
def stats1():
  global tottim
  print('Stats')
  averagetimeperclient = tottim / clientesAfterService
  print('The average time per client: %.3f' %(averagetimeperclient))

In [298]:
stats1()

Stats
The average time per client: 126.990


In [307]:
mu = 40
sigma = 10
  
# using the lognormvariate() method
print(random.lognormvariate(mu, sigma))

4234172899925373.5
